# SP 8, "Existence": Problem 1

Let $S$ be a set of binary strings $a_{1}...a_{n}$ of length $n$ (where juxtaposition means concatenation). We call $S$ $k$-complete if for any indices $1 \leq i_{1} < ··· < i_{k} \leq n$ and any binary string $b_{1}...b_{k}$ of length $k$, there is a string $s_{1}...s_{n}$ in $S$ such that $s_{i1}s_{i2}...s_{ik} = b_{1}b_{2} ...b_{k}$. For example, for $n = 3$, the set $S = \{001, 010, 011, 100, 101, 110\}$ is $2$-complete since all $4$ patterns of $0$’s and $1$’s of length $2$ can be found in any $2$ positions. 

Show that **if $\binom{n}{k}2^{k}(1-2^{-k})^{m} < 1$, then there exists a $k$-complete set of size at most $m$**.

In [1]:
from itertools import product, combinations, tee, chain
from operator import itemgetter
from math import comb

## Intuition & Strategy

We see that there indeed exists a $k$-complete set of size $\leq m$ from the example above: $\left|S\right| = 6 \leq m = 9$ (from solving for $m$ in the condition above). The general strategy is:

1. Let $X$ be the size / count the elements of a $k$-complete set for a given $n \geq k \in \{0, 1, ...\}$.
2. If we can show $E(X) \leq m$, then we know $\exists S^{\prime}$ where $S^{\prime}$ is a set of size at most $m$.

Note: The below is overkill re: memory efficiency -- it's primarily practice writing classes and using generators in Python. The processing is very slow because everything is lazily evaluated (e.g., power set checking)!

In [2]:
# Write a class that defines BinStrings object defined by n, k
class BinStrings():
    """
    Binary strings of length n that are determined to be k-complete or not
    """
                
    def __init__(self, n, k):
        self.n = n
        self.k = k
    
    @staticmethod
    def perm(seq, l):
        """
        Returns generator for all permutations (of `seq`) of length `l` with replacement
        one element at a time
        """
        for p in product(seq, repeat=l):
            yield p

    @staticmethod
    def power_set(iterable):
        """
        Returns power set of iterable
        """
        s = set(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    
    def __bin_str_indexer(self):
        """
        Return index combinations (length k) to check each set in S
        """
        for idx in combinations(range(self.n), self.k):
            yield idx

    def __k_complete_check(self, S):
        """
        Returns boolean of whether set S is k-complete
        """
        conds_met = 0
        for s in S:
            for idx in self.__bin_str_indexer(): 
                for bin_str in self.perm([0, 1], self.k):
                    b_at_s_idx = itemgetter(*idx)(s)
                    if b_at_s_idx == bin_str or (b_at_s_idx,) == bin_str:
                        conds_met += 1
                        
        return conds_met >= comb(self.n, self.k) * (2 ** self.k)

    def expected_k_complete_size(self):
        """
        Calculate E(X) where X is size of k-complete set S given n, k
        """
        size, total = 0, 0
        for S in self.power_set(self.perm([0, 1], self.n)):
            if self.__k_complete_check(S):
                total += 1
                size += len(S)
        try:
            return size / total
        except ZeroDivisionError:
            print('No sets are k-complete!')
            return 0
            
    def m_limit(self):
        """
        Returns upper bound on |S|
        """
        cond_less_than_1, m = False, 0
        while not cond_less_than_1:
            m += 1
            cond_less_than_1 = comb(self.n, self.k) * (2 ** self.k) * ((1 - (2 ** -self.k)) ** m) < 1
        return m

We can see pretty clearly that $E(X_{n=3, k=2}) \approx 4.86 \leq m = 9$, which means there exists a $k$-complete set of at most $m = 9$ elements.

In [3]:
test_config = BinStrings(3, 2)

# Try example case
print(f"E(X) for n = {test_config.n}, k = {test_config.k}: {test_config.expected_k_complete_size():.5f}")
print(f"Set size limit for n = {test_config.n}, k = {test_config.k}: {test_config.m_limit()}")

E(X) for n = 3, k = 2: 4.85890
Set size limit for n = 3, k = 2: 9


The base case, when $n=1, k=1$, is why the problem states "...at most size $m$").

In [4]:
%%time

# Print outcomes for a range of n, k; this blows up when n = 5 or more
for n in range(1, 4 + 1):
    for k in range(1, n + 1):
        test_config = BinStrings(n, k)
        print(f"E(X) for n = {test_config.n}, k = {test_config.k}: {test_config.expected_k_complete_size():.5f}")
        print(f"Set size limit for n = {test_config.n}, k = {test_config.k}: {test_config.m_limit()}")

E(X) for n = 1, k = 1: 2.00000
Set size limit for n = 1, k = 1: 2
E(X) for n = 2, k = 1: 2.54545
Set size limit for n = 2, k = 1: 3
E(X) for n = 2, k = 2: 4.00000
Set size limit for n = 2, k = 2: 5
E(X) for n = 3, k = 1: 4.11336
Set size limit for n = 3, k = 1: 3
E(X) for n = 3, k = 2: 4.85890
Set size limit for n = 3, k = 2: 9
E(X) for n = 3, k = 3: 8.00000
Set size limit for n = 3, k = 3: 16
E(X) for n = 4, k = 1: 8.00183
Set size limit for n = 4, k = 1: 4
E(X) for n = 4, k = 2: 8.05614
Set size limit for n = 4, k = 2: 12
E(X) for n = 4, k = 3: 9.31316
Set size limit for n = 4, k = 3: 26
E(X) for n = 4, k = 4: 16.00000
Set size limit for n = 4, k = 4: 43
CPU times: user 18 s, sys: 26.2 ms, total: 18.1 s
Wall time: 18.1 s
